In [2]:
# Local clone
! git clone https://github.com/nanopiero/CML_processing_by_ML.git

Cloning into 'CML_processing_by_ML'...
remote: Enumerating objects: 317, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 317 (delta 73), reused 10 (delta 10), pack-reused 207 (from 1)
Receiving objects: 100% (317/317), 11.15 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [3]:
# Imports
from os.path import join, isdir, isfile
from os import listdir as ls
import copy
import torch
import numpy as np

import sys
sys.path.append('CML_processing_by_ML')

from src.utils.simulation import create_dataloader
import src.utils.architectures_fcn
from src.utils.architectures import load_archi
from src.utils.architectures_fcn import UNet_causal_5mn_atrous, UNet_causal_5mn_atrous_rescale

In [4]:
# Dictionary with pseudo "distances" (distances between two antennnas) for 1,000 pseudo CML ids.
idx2distance = {i: 0.2 +  1.8 * torch.rand((1,)).item() for i in range(0, 1000)}
duration = 4096  # length of the time series
batch_size = 100  # Number of samples per batch
dataloader = create_dataloader(duration, idx2distance, batch_size)

In [4]:
# Here we samples 100 ground-truth rainy processes and their noisy counterpart
# A rainy process is modeled by a 1-d Neymann-Scott process
# The Intensity of the Poisson process for parent events is 0.05 x distance
# The resulting rainy process is divided by the distance to give the "ground truth"
# while it is corrupted through the following steps to yield "noisy_series":
# - applying a non linear conversion to an attenuation in db
# - applying a "wet antenna convolution filter" (kind of sliding mean)
# - adding a high (gaussian noise with non linear dependance of sigma wrt the intensity)
# - adding a low frequency random processes

for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
  if batch_idx == 0:
    break

In [ ]:
import matplotlib.pyplot as plt
sigma = 2
for k in range(5):
  print(idxs[k], dists[k])
  plt.figure(figsize=(10, 6))
  plt.plot(np.arange(duration), ground_truths[k], label='ground_truth')
  plt.plot(np.arange(duration), noisy_series[k], label='predictor')
  plt.title(f'Inputs and Targets for the CML n°{idxs[k].item():.0f} (ditance: {dists[k].item():.2f})')
  plt.xlabel('Time (minutes)')
  plt.ylabel('Event Density')
  plt.ylim(-1,6)
  plt.legend()
  plt.show()

In [ ]:
# Same weights for all CMLs

arch = "UNet_causal_5mn_atrous"
nchannels = 1
nclasses = 1 # Regression only
dilation = 2
atrous_rates=[6, 12, 18] #, 24, 30, 36, 42]
additional_parameters = 0

model = load_archi(arch, nchannels, nclasses, size=64, dilation=1,
                   atrous_rates=atrous_rates, fixed_cumul=False,
                   additional_parameters=additional_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# model = UNet(1, 1, 16).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

num_epochs = 300  # Adjust based on your needs

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(model, data_loader, num_samples=1):
    model.eval()
    L = 1000
    with torch.no_grad():
        for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):

            inputs = noisy_series.to(device).unsqueeze(1).float()  # Adjust input dimensions
            outputs = model(inputs).cpu()

            for i in range(num_samples):
                plt.figure(figsize=(16, 4))
                # plt.plot(noisy_series[i].squeeze(), label='input')
                plt.plot(ground_truths[i,:L].squeeze(), label='Observation')
                plt.plot(outputs[i].squeeze()[:L], label='Prediction', linestyle='--')
                plt.legend()
                plt.show()
            break  # Just show the first batch

visualize_predictions(model, dataloader, num_samples=5)


In [ ]:
# How to do better ?

In [ ]:
# sol 1 : UNet_causal_5mn_atrous_rescale: adds a scaling parameter for each CML

arch = "UNet_causal_5mn_atrous_rescale"
nchannels = 1
nclasses = 1 # Regression only
dilation = 2
atrous_rates=[6, 12, 18] #, 24, 30, 36, 42]
additional_parameters = 1005

model = load_archi(arch, nchannels, nclasses, size=64, dilation=1,
                   atrous_rates=atrous_rates, fixed_cumul=False,
                   additional_parameters=additional_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# model = UNet(1, 1, 16).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

num_epochs = 50  # Adjust based on your needs

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        outputs, p = model(inputs, indices=idxs.to(device))
        outputs[:,:,:] *= p[5:].view(outputs.shape[0],1,1)

        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

In [44]:
# sol 2 (step 1): with UNet_causal_5mn_multiplicative_rescale,
# the scaling parameter
# is yielded by specific perceptrons (one per CML)

arch = "UNet_causal_5mn_atrous_multiplicative_rescale"
nchannels = 1
nclasses = 1
dilation = 2
atrous_rates=[6, 12, 18] #, 24, 30, 36, 42]
additional_parameters = 0

model = load_archi(arch, nchannels, nclasses, size=64, dilation=1,
                   atrous_rates=atrous_rates, fixed_cumul=False,
                   additional_parameters=additional_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# model = UNet(1, 1, 16).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# model.freeze_specific_parts()

criterion = torch.nn.MSELoss()

num_epochs = 50  # Adjust based on your needs

model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        use_first_network = torch.rand(idxs.shape, device=inputs.device) > 0.75
        idxs[use_first_network] = -1
        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, idxs.to(device))

        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

Epoch [1/50], Loss: 0.4691
Epoch [2/50], Loss: 0.2925
Epoch [3/50], Loss: 0.2658
Epoch [4/50], Loss: 0.2584
Epoch [5/50], Loss: 0.2501
Epoch [6/50], Loss: 0.2469
Epoch [7/50], Loss: 0.2453
Epoch [8/50], Loss: 0.2398
Epoch [9/50], Loss: 0.2292
Epoch [10/50], Loss: 0.2239
Epoch [11/50], Loss: 0.2164
Epoch [12/50], Loss: 0.2123
Epoch [13/50], Loss: 0.2104
Epoch [14/50], Loss: 0.2105
Epoch [15/50], Loss: 0.2027
Epoch [16/50], Loss: 0.1889
Epoch [17/50], Loss: 0.1720
Epoch [18/50], Loss: 0.1645
Epoch [19/50], Loss: 0.1664
Epoch [20/50], Loss: 0.1620
Epoch [21/50], Loss: 0.1566
Epoch [22/50], Loss: 0.1540
Epoch [23/50], Loss: 0.1541
Epoch [24/50], Loss: 0.1507
Epoch [25/50], Loss: 0.1479
Epoch [26/50], Loss: 0.1470
Epoch [27/50], Loss: 0.1454
Epoch [28/50], Loss: 0.1445
Epoch [29/50], Loss: 0.1461
Epoch [30/50], Loss: 0.1437
Epoch [31/50], Loss: 0.1431
Epoch [32/50], Loss: 0.1445
Epoch [33/50], Loss: 0.1423
Epoch [34/50], Loss: 0.1401
Epoch [35/50], Loss: 0.1400
Epoch [36/50], Loss: 0.1386
E

In [45]:
# sol 2 (step 2): the generic part is freezed while the specific perceptrons
# are fine tuned
num_epochs = 20

model.unfreeze_specific_parts()
model.freeze_generic_parts()

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs, idxs.to(device))

        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

Epoch [1/20], Loss: 0.1253
Epoch [2/20], Loss: 0.1252
Epoch [3/20], Loss: 0.1245
Epoch [4/20], Loss: 0.1247
Epoch [5/20], Loss: 0.1254
Epoch [6/20], Loss: 0.1243
Epoch [7/20], Loss: 0.1239
Epoch [8/20], Loss: 0.1238
Epoch [9/20], Loss: 0.1242
Epoch [10/20], Loss: 0.1243
Epoch [11/20], Loss: 0.1254
Epoch [12/20], Loss: 0.1240
Epoch [13/20], Loss: 0.1240
Epoch [14/20], Loss: 0.1242
Epoch [15/20], Loss: 0.1238
Epoch [16/20], Loss: 0.1243
Epoch [17/20], Loss: 0.1236
Epoch [18/20], Loss: 0.1243
Epoch [19/20], Loss: 0.1247
Epoch [20/20], Loss: 0.1238


In [ ]:
# Now, let's suppose that the target itself is noisy
# and that's the case when we want to use as a target
# the reference we wish to improve.
# Here, hence, the reference is no more the ground truth RR(t) but
# its randomly-time-shifted version, that is to say,
# RR(t + delta_t), the shift delta_t slowly varying from -2 time steps to 2 time steps

# Note that the new targets are not centered on the ground truth value
# So MSE won't give the best guess (nor MAE and other standard losses)
# However, the conditional expectancy of the target knowing the ref
# is a strictly growing function of the ground truth
# so it ends up with a problem of calibration.
# As the noisy process doesn't impair the distribution, a simple
# QQ-plot will serve as calibration curve.

In [7]:
def apply_random_shifts(inputs):
    device = inputs.device
    bs = inputs.shape[0]
    N = inputs.shape[1]
    a = torch.arange(N, device=device).unsqueeze(0)
    b = 100 + 400*torch.rand(bs, 1, device=device)
    sin1 = torch.sin( 2 * torch.pi / (b * (1 + torch.rand(bs, 1, device=device))) \
                     * a + torch.rand(bs, 1, device=device))
    sin2 = torch.sin( 2 * torch.pi / (b * (1 + 2*torch.rand(bs, 1, device=device))) \
                     * a + torch.rand(bs, 1, device=device))
    sin3 = torch.sin( 2 * torch.pi / (b * (1 + 2*torch.rand(bs, 1, device=device))) * a \
                     + torch.rand(bs, 1, device=device))
    x = sin1 + sin2 + sin3
    y = 1*(x > 0.2) + 1*(x > 0.5) + 1*(x > 0.8) + 1*(x > 1.1) + 1*(x > 1.5) \
      - 1*(x < -0.8) - 1*(x < -1.1)  - 1*(x < -1.5)  - 1*(x < -0.2)

    y[:, 0:10] = 0
    y[:, -10:] = 0
    return torch.gather(inputs, 1,  a + y)

In [ ]:
# show the shifts
import matplotlib.pyplot as plt

for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
  if batch_idx == 0:
    break

reference = apply_random_shifts(ground_truths)
print(ground_truths.shape, reference.shape)
for k in range(5):
  print(idxs[k], dists[k])
  plt.figure(figsize=(20, 6))
  plt.plot(np.arange(duration)[0:360], ground_truths[k][0:360], label='ground_truth')
  plt.plot(np.arange(duration)[0:360], reference[k][0:360], label='targets')
  plt.title(f'Ground truth and Targets for the CML n°{idxs[k].item():.0f} (ditance: {dists[k].item():.2f})')
  plt.xlabel('Time (minutes)')
  plt.ylabel('Event Density')
  plt.ylim(-1,6)
  plt.legend()
  plt.show()

In [47]:
# training the sol 2 with reference (time shifted ground truth) as target

arch = "UNet_causal_5mn_atrous_multiplicative_rescale"
nchannels = 1
nclasses = 1
dilation = 2
atrous_rates=[6, 12, 18] #, 24, 30, 36, 42]
additional_parameters = 0

model = load_archi(arch, nchannels, nclasses, size=64, dilation=1,
                   atrous_rates=atrous_rates, fixed_cumul=False,
                   additional_parameters=additional_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# model = UNet(1, 1, 16).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# model.freeze_specific_parts()

criterion = torch.nn.MSELoss()

num_epochs = 50  # Adjust based on your needs

model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    running_loss2 = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, ground_truths = noisy_series.to(device), \
                                ground_truths.to(device)

        # Here we apply a random time shift
        targets = apply_random_shifts(ground_truths)

        use_first_network = torch.rand(idxs.shape, device=inputs.device) > 0.75
        idxs[use_first_network] = -1

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)
        ground_truths = ground_truths.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, idxs.to(device))

        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        with torch.no_grad():
          loss2 = criterion(outputs, ground_truths)
        running_loss2 += loss2.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss vs targets: {running_loss/len(dataloader):.4f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss vs ground truth: {running_loss2/len(dataloader):.4f}')


print('step 2 - fine tuning')
num_epochs = 20

model.unfreeze_specific_parts()
model.freeze_generic_parts()

for epoch in range(num_epochs):
    running_loss = 0.0
    running_loss2 = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, ground_truths = noisy_series.to(device), \
                                ground_truths.to(device)

        # Here we apply a random time shift
        targets = apply_random_shifts(ground_truths)

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)
        ground_truths = ground_truths.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs, idxs.to(device))

        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        with torch.no_grad():
          loss2 = criterion(outputs, ground_truths)
        running_loss2 += loss2.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss vs targets: {running_loss/len(dataloader):.4f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss vs ground truth: {running_loss2/len(dataloader):.4f}')


Epoch [1/50], Loss vs targets: 0.5193
Epoch [1/50], Loss vs ground truth: 0.4761
Epoch [2/50], Loss vs targets: 0.3665
Epoch [2/50], Loss vs ground truth: 0.2955
Epoch [3/50], Loss vs targets: 0.3477
Epoch [3/50], Loss vs ground truth: 0.2724
Epoch [4/50], Loss vs targets: 0.3404
Epoch [4/50], Loss vs ground truth: 0.2620
Epoch [5/50], Loss vs targets: 0.3322
Epoch [5/50], Loss vs ground truth: 0.2511
Epoch [6/50], Loss vs targets: 0.3266
Epoch [6/50], Loss vs ground truth: 0.2451
Epoch [7/50], Loss vs targets: 0.3228
Epoch [7/50], Loss vs ground truth: 0.2414
Epoch [8/50], Loss vs targets: 0.3152
Epoch [8/50], Loss vs ground truth: 0.2323
Epoch [9/50], Loss vs targets: 0.3178
Epoch [9/50], Loss vs ground truth: 0.2324
Epoch [10/50], Loss vs targets: 0.3129
Epoch [10/50], Loss vs ground truth: 0.2261
Epoch [11/50], Loss vs targets: 0.3018
Epoch [11/50], Loss vs ground truth: 0.2156
Epoch [12/50], Loss vs targets: 0.2909
Epoch [12/50], Loss vs ground truth: 0.2041
Epoch [13/50], Loss vs

In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(model, data_loader, num_samples=1):
    model.eval()
    L = 1000
    with torch.no_grad():
        for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
            reference = apply_random_shifts(ground_truths)
            inputs = noisy_series.to(device).unsqueeze(1).float()  # Adjust input dimensions
            outputs = model(inputs, idxs.to(device)).cpu()

            for i in range(num_samples):
                plt.figure(figsize=(16, 4))
                # plt.plot(noisy_series[i].squeeze(), label='input')
                plt.plot(ground_truths[i,:L].squeeze(), label='Observation')
                plt.plot(outputs[i].squeeze()[:L], label='Prediction', linestyle='--')
                plt.plot(reference[i,:L].squeeze(), label='Reference')

                plt.legend()
                plt.show()
            break  # Just show the first batch

visualize_predictions(model, dataloader, num_samples=5)

In [ ]:
for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
  if batch_idx == 0:
    break

    reference = apply_random_shifts(ground_truths)
    inputs = noisy_series.to(device).unsqueeze(1).float()  # Adjust input dimensions
    outputs = model(inputs, idxs.to(device)).cpu()

plt.scatter(np.quantile(zm_pred[np.logical_not(np.isnan(zm_pred))], np.arange(0,1,0.01)), np.quantile(vv_obs,np.arange(0,1,0.01)))


In [ ]:
# calibrate the prediction

In [ ]:
# display a QQ plot of reference / prediction / calibrated prediction against ground-thruth

In [ ]:
#Display the results

In [ ]:
# To pull and reload, if needed:
! cd CML_processing_by_ML ; git pull ; cd ..

import importlib
importlib.reload(src.utils.architectures_fcn)
importlib.reload(src.utils.architectures)
from src.utils.architectures_fcn import  UNet_causal_5mn_atrous_multiplicative_rescale
from src.utils.architectures import  load_archi